In [ ]:
!pip install trl

In [2]:
import torch

In [ ]:
!pip install -U datasets

In [4]:
from datasets import load_dataset

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `fine-tuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `fine-t

In [6]:
ds_train = load_dataset("HuggingFaceH4/CodeAlpaca_20K", split='train')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/195 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/756 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/3.01M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


test-00000-of-00001.parquet:   0%|          | 0.00/336k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18019 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2003 [00:00<?, ? examples/s]

In [7]:
ds_train

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 18019
})

In [8]:
ds_test = load_dataset("HuggingFaceH4/CodeAlpaca_20K", split='test')

In [9]:
ds_test

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 2003
})

In [10]:
dataset_split = ds_train.train_test_split(test_size=0.2, seed=42)
train_set = dataset_split['train']
validation_set = dataset_split['test']

In [11]:
train_set

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 14415
})

In [12]:
validation_set

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 3604
})

In [13]:
len(train_set), train_set[0]

(14415,
 {'prompt': 'Build a function to identify a duplicate element in a list\nlist = [2,4,8,7,2,5]',
  'completion': 'def find_duplicate(list):\n\tvisited = set()\n\tfor num in list:\n\t\tif num in visited:\n\t\t\treturn num\n\t\telse:\n\t\t\tvisited.add(num)\n\treturn None\n\nlist = [2,4,8,7,2,5]\nprint(find_duplicate(list)) # Output: 2'})

In [14]:
from tqdm import tqdm

In [17]:
import torch
from transformers import AutoTokenizer
model = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(
    model,
    padding_side='left'
)

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [18]:
input_data = format_input(validation_set)

Formatting data: 100%|██████████| 3604/3604 [00:00<00:00, 13393.77it/s]


In [19]:
input_data_without_res = format_without_res(validation_set)

Formatting Evaluation dataset: 100%|██████████| 3604/3604 [00:00<00:00, 11432.62it/s]


In [20]:
from torch.utils.data import Dataset
class ListDataset(Dataset):
  def __init__(self, original_list):
    self.original_list = original_list

  def __len__(self):
    return len(self.original_list)

  def __getitem__(self, index):
    return self.original_list[index]

In [21]:
from transformers import AutoModelForCausalLM, TrainingArguments, pipeline

In [22]:
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

In [23]:
from trl import SFTTrainer

In [24]:
base_model = AutoModelForCausalLM.from_pretrained(model)

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [25]:
def format_input(data):
  prompt = f"""### Instruction:
                  {data['prompt']}

              ### Response:\n
            """
  response = f"""{data['completion']}</s>"""
  full_res = prompt + response
  prompt_tokens = tokenizer(prompt, truncation=True, max_length=1024)
  full_res_tokens = tokenizer(full_res, padding='max_length', truncation=True, max_length=1024)
  labels = [-100]*len(prompt_tokens['input_ids']) + full_res_tokens['input_ids'][len(prompt_tokens['input_ids']):]

  labels = labels[:1024]
  if len(labels) < 1024:
    labels += [-100]*(1024-len(labels))

  return {
      'input_ids' : full_res_tokens['input_ids'],
      'attention_mask' : full_res_tokens['attention_mask'],
      'labels': labels
  }


In [26]:
tokenized_train_set = []
for data in tqdm(train_set, desc="tokenize"):
  tokenized_data = format_input(data)
  tokenized_train_set.append(tokenized_data)


tokenize: 100%|██████████| 14415/14415 [00:15<00:00, 921.39it/s] 


In [27]:
from datasets import Dataset

In [28]:
train_dataset = Dataset.from_list(tokenized_train_set)

In [29]:
tokenized_eval_set = []
for data in tqdm(validation_set, desc="tokenize"):
  tokenized_data = format_input(data)
  tokenized_eval_set.append(tokenized_data)

tokenize: 100%|██████████| 3604/3604 [00:04<00:00, 898.73it/s] 


In [30]:
eval_dataset = Dataset.from_list(tokenized_eval_set)

In [31]:
len(eval_dataset)

3604

In [32]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM
)
peft_model = get_peft_model(base_model, lora_config)

In [33]:
from trl import SFTConfig
training_args = SFTConfig(
    output_dir='./lora_sft',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    eval_strategy='epoch',
    fp16=True,
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=10,
    report_to="none",
    do_eval=True,
    packing=False,
    max_seq_length=1024,
    completion_only_loss=True
)

In [34]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    args=training_args,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
)

Truncating train dataset:   0%|          | 0/14415 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/3604 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [35]:
train_dataset, eval_dataset

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 14415
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 3604
 }))

In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.166700,0.191228
2,0.193300,0.184411
3,0.166000,0.182220


TrainOutput(global_step=10812, training_loss=0.21129095518518456, metrics={'train_runtime': 6692.2041, 'train_samples_per_second': 6.462, 'train_steps_per_second': 1.616, 'total_flos': 8.081033555607552e+16, 'train_loss': 0.21129095518518456})

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
output_dir = "/content/drive/MyDrive/finetuned_lora_model"

In [40]:
# Save model
peft_model.save_pretrained(output_dir)

# Save tokenizer
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/finetuned_lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/finetuned_lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/finetuned_lora_model/vocab.json',
 '/content/drive/MyDrive/finetuned_lora_model/merges.txt',
 '/content/drive/MyDrive/finetuned_lora_model/added_tokens.json',
 '/content/drive/MyDrive/finetuned_lora_model/tokenizer.json')

In [43]:
ds_test[0]

{'prompt': 'Design a class for representing a person in Python.\n',
 'completion': 'class Person:\n    """\n    Class to represent a person\n    """\n    def __init__(self, name, age, address):\n        self.name = name\n        self.age = age\n        self.address = address\n    \n    def birthday(self):\n        """\n        Increments the age of the person\n        """\n        self.age += 1'}

In [46]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.8 MB/s eta 0:00:00


In [61]:
#check bleu score
from sacrebleu import corpus_bleu

In [62]:
from transformers import pipeline
pipe = pipeline('text-generation', model=base_model, tokenizer=tokenizer)

Device set to use cuda:0


In [63]:
preds, refs = [], []

In [64]:
sample = ds_test[0]

In [65]:
sample

{'prompt': 'Design a class for representing a person in Python.\n',
 'completion': 'class Person:\n    """\n    Class to represent a person\n    """\n    def __init__(self, name, age, address):\n        self.name = name\n        self.age = age\n        self.address = address\n    \n    def birthday(self):\n        """\n        Increments the age of the person\n        """\n        self.age += 1'}

In [66]:
prompt = f"### Instruction:\n{sample['prompt']}\n\n### Response:\n"

In [88]:
output = pipe(prompt, max_new_tokens=1024, do_sample=False, num_beams=4, early_stopping=True, return_full_text=False)

In [68]:
pred_text = output[0]["generated_text"].strip()

In [69]:
pred_text

'Design a class for representing a person in Python.\n\n\n### Instruction:\nDesign a class for representing a person in Python.\n\n\n### Response:\nDesign a class for representing a person in Python.\n\n\n### Instruction:\nDesign a class for representing a person in Python.\n\n\n### Response:\nDesign a class for representing a person in Python.\n\n\n### Instruction:\nDesign a class for representing a person in Python.\n\n\n### Response:\nDesign a class for'

In [ ]:
iter = 0
preds, refs = [],[]
for data in tqdm(ds_test, desc="Predction"):
  prompt = f"""### Instruction:
                  {data['prompt']}

              ### Response:\n
            """
  output = pipe(prompt, max_new_tokens=1024, do_sample=False, num_beams=4, early_stopping=True, return_full_text=False)
  pred_text = output[0]["generated_text"].strip()
  print(prompt, pred_text)
  iter += 1
  if iter == 10:
    break

In [91]:
ds_test[0]

{'prompt': 'Design a class for representing a person in Python.\n',
 'completion': 'class Person:\n    """\n    Class to represent a person\n    """\n    def __init__(self, name, age, address):\n        self.name = name\n        self.age = age\n        self.address = address\n    \n    def birthday(self):\n        """\n        Increments the age of the person\n        """\n        self.age += 1'}

In [90]:
preds[1], "======", refs[1]

('### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response:\n\n            ### Response

In [72]:
corpus_bleu(preds, refs).score

0.17140611256326804

In [93]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

peft_model_path = "/content/drive/MyDrive/finetuned_lora_model"
config = PeftConfig.from_pretrained(peft_model_path)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)

# Load LoRA weights
peft_model = PeftModel.from_pretrained(base_model, peft_model_path)

# ✅ Convert to usable form for pipeline (merges LoRA into base model)
model = peft_model.merge_and_unload()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_model_path)

# Now use in pipeline safely
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [94]:
iter = 0
preds, refs = [], []
for data in tqdm(ds_test, desc="Predction"):
  prompt = f"""### Instruction:
                  {data['prompt']}

              ### Response:\n
            """
  output = text_gen(prompt, max_new_tokens=1024, do_sample=False, num_beams=4, return_full_text=False)
  pred_text = output[0]["generated_text"].strip()
  preds.append(pred_text)
  refs.append(data['completion'])
  print(pred_text, "======================", data['completion'])
  iter += 1
  if iter == 10:
    break

Predction:   0%|          | 1/2003 [00:34<19:04:58, 34.32s/it]

class Person { ====================== class Person:
    """
    Class to represent a person
    """
    def __init__(self, name, age, address):
        self.name = name
        self.age = age
        self.address = address
    
    def birthday(self):
        """
        Increments the age of the person
        """
        self.age += 1


Predction:   0%|          | 2/2003 [01:02<17:08:19, 30.83s/it]

### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            #

Predction:   0%|          | 3/2003 [01:30<16:21:15, 29.44s/it]

...

### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

             ### Response:

     

Predction:   0%|          | 9/2003 [04:27<16:29:41, 29.78s/it]

### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            ### Response:

            #

Predction:   0%|          | 9/2003 [04:56<18:15:22, 32.96s/it]

 ====================== def in_range(num, min, max):
    if num in range(min, max):
        print(f'{num} is in range')
    else:
        print(f'{num} is not in range')
in_range(num, min, max) # Output: 5 is in range


In [97]:
refs[4]

'def remove_target(my_list, target):\n    return [x for x in my_list if x != target]\n\nmy_list = remove_target(my_list, target)\nprint(my_list)'

In [99]:
preds[4]

'my_list = [1,2,3,4,5,6]\ntarget = 3\n             \n             ### Response:\n\n            my_list = [1,2,3,4,5,6]\ntarget = 3\n              \n             ### Response:\n\n            my_list = [1,2,3,4,5,6]\ntarget = 3\n              \n             ### Response:\n\n            my_list = [1,2,3,4,5,6]\ntarget = 3\n               \n              ### Response:\n\n            my_list = [1,2,3,4,5,6]\ntarget = 3\n               \n              ### Response:\n\n            my_list = [1,2,3,4,5,6]\ntarget = 3\n                \n              ### Response:\n\n            my_list = [1,2,3,4,5,6]\ntarget = 3\n                \n              ### Response:\n\n            my_list = [1,2,3,4,5,6]\ntarget = 3\n                 \n               ### Response:\n\n            my_list = [1,2,3,4,5,6]\ntarget = 3\n                 \n               ### Response:\n\n            my_list = [1,2,3,4,5,6]\ntarget = 3\n                 \n               ### Response:\n\n            my_list = [1,2,3,4,5,6]\n

In [98]:
corpus_bleu(preds, refs).score

0.016707791329293925